In [3]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import folium
import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

# Process Bridge Data
Load and consolidate SHN bridge data points with small buffer + dissolve, then apply larger analysis buffer

In [4]:
# load bridges
# In terminal:
'''
pip install esridump
esri2geojson https://caltrans-gis.dot.ca.gov/arcgis/rest/services/CHhighway/State_Highway_Bridges/FeatureServer/0 bridges.geojson
'''

bridges = gpd.read_file("./bridges.geojson")
    
shared_utils.utils.geoparquet_gcs_export(bridges, GCS_FILE_PATH, "shn_bridges")

In [5]:
bridges_buffer = bridges.copy()
bridges_buffer.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
bridges_buffer = bridges_buffer.to_crs(3310)

In [8]:
# use 100 meter buffer (freeway width?)
bridges_buffer.geometry = bridges_buffer.buffer(100)

In [9]:
bridges_buffer.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13201 entries, 0 to 13200
Data columns (total 36 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID_1  13201 non-null  int64   
 1   OBJECTID    13201 non-null  int64   
 2   DIST        13201 non-null  int64   
 3   CO          13201 non-null  object  
 4   BRIDGE      13201 non-null  object  
 5   BRIDGE_X    13201 non-null  float64 
 6   BRIDGE_Y    13201 non-null  float64 
 7   LAT         13201 non-null  float64 
 8   LON         13201 non-null  float64 
 9   NAME        13201 non-null  object  
 10  LOC         13201 non-null  object  
 11  YRBLT       13201 non-null  int64   
 12  HST         13201 non-null  int64   
 13  FAC         13201 non-null  object  
 14  APWID       13201 non-null  float64 
 15  LENG        13201 non-null  float64 
 16  DK_AREA     13201 non-null  int64   
 17  LSW         13201 non-null  int64   
 18  RSW         13201 non-null  int64   
 

In [14]:
# keep only certain columns
bridges_buffer_select = bridges_buffer[['geometry','DIST', 'CO', 'BRIDGE', 'NAME', 'FAC', 'INTERSEC', 'AADT', 'PM', 'RTE']]

In [15]:
# try dissolving it by sjoining it to itself: https://stackoverflow.com/questions/73566774/group-by-and-combine-intersecting-overlapping-geometries-in-geopandas
bridges_intersect = bridges_buffer_select.sjoin(bridges_buffer_select, how="left", predicate="intersects")

# dissolve intersections on right bridge ID using the minimum value
bridges_intersect_dissolve = bridges_intersect.dissolve("BRIDGE_right", aggfunc="min")

# dissolve again on left bridge ID using minimum
bridges_intersect_dissolve = bridges_intersect_dissolve.reset_index().dissolve("BRIDGE_left", aggfunc="min")

In [17]:
len(bridges_intersect_dissolve)

8972

In [18]:
bridges_intersect_dissolve.head(10)

,geometry,BRIDGE_right,DIST_left,CO_left,NAME_left,FAC_left,INTERSEC_left,AADT_left,PM_left,RTE_left,index_right,DIST_right,CO_right,NAME_right,FAC_right,INTERSEC_right,AADT_right,PM_right,RTE_right
BRIDGE_left,,,,,,,,,,,,,,,,,,,
01 0002,"POLYGON ((-338521.565 400172.680, -338522.046 ...",01 0002,1,DN,MINOT CREEK,U.S. HIGHWAY 101,MINOT CREEK,4600,8.14,101,0,1,DN,MINOT CREEK,U.S. HIGHWAY 101,MINOT CREEK,4600,8.14,101
01 0003,"POLYGON ((-338835.338 400682.335, -338835.820 ...",01 0003,1,DN,HUNTER CREEK,U.S. HIGHWAY 101,HUNTER CREEK,4600,8.51,101,1,1,DN,HUNTER CREEK,U.S. HIGHWAY 101,HUNTER CREEK,4600,8.51,101
01 0004,"POLYGON ((-339622.423 401838.968, -339622.904 ...",01 0004,1,DN,HIGH PRAIRIE CREEK,U.S. HIGHWAY 101,HIGH PRAIRIE CREEK,4600,9.39,101,2,1,DN,HIGH PRAIRIE CREEK,U.S. HIGHWAY 101,HIGH PRAIRIE CREEK,4600,9.39,101
01 0005,"POLYGON ((-342103.890 405963.047, -342104.372 ...",01 0005,1,DN,WILSON CREEK (LOUIS P DEMARTIN SR MEMORIAL BRI...,U.S. HIGHWAY 101,WILSON CREEK,4700,12.64,101,3,1,DN,WILSON CREEK (LOUIS P DEMARTIN SR MEMORIAL BRI...,U.S. HIGHWAY 101,WILSON CREEK,4700,12.64,101
01 0006,"POLYGON ((-339619.788 428312.565, -339620.270 ...",01 0006,1,DN,SMITH RIVER (HIOUCHI BRIDGE),U.S. HIGHWAY 199,SMITH RIVER,4300,4.22,199,4,1,DN,SMITH RIVER (HIOUCHI BRIDGE),U.S. HIGHWAY 199,SMITH RIVER,4300,4.22,199
01 0007,"POLYGON ((-337416.933 427622.228, -337417.415 ...",01 0007,1,DN,MYRTLE CREEK,U.S. HIGHWAY 199,MYRTLE CREEK,4300,7.09,199,5,1,DN,MYRTLE CREEK,U.S. HIGHWAY 199,MYRTLE CREEK,4300,7.09,199
01 0009,"POLYGON ((-333578.563 432020.993, -333579.044 ...",01 0009,1,DN,SMITH RIVER (MARY ADAMS PEACOCK MEMORIAL BRIDGE),U.S. HIGHWAY 199,SMITH RIVER,3750,R11.95,199,6,1,DN,SMITH RIVER (MARY ADAMS PEACOCK MEMORIAL BRIDGE),U.S. HIGHWAY 199,SMITH RIVER,3750,R11.95,199
01 0012,"POLYGON ((-323042.968 433257.151, -323043.449 ...",01 0012,1,DN,MIDDLE FORK SMITH RIVER,U.S. HIGHWAY 199,MIDDLE FORK SMITH RIVER,3100,R19.22,199,7,1,DN,MIDDLE FORK SMITH RIVER,U.S. HIGHWAY 199,MIDDLE FORK SMITH RIVER,3100,R19.22,199
01 0014,"POLYGON ((-319470.435 434993.491, -319470.916 ...",01 0014,1,DN,PATRICK CREEK,U.S. HIGHWAY 199,PATRICK CREEK,3750,R22.07,199,8,1,DN,PATRICK CREEK,U.S. HIGHWAY 199,PATRICK CREEK,3750,R22.07,199


In [19]:
# buffer another 100m for analysis of crashes/encampments
bridges_intersect_dissolve.geometry = bridges_intersect_dissolve.buffer(100)

In [23]:
# drop extra vars
bridges_intersect_dissolve.drop(bridges_intersect_dissolve.loc[:, 'index_right':'RTE_right'], axis = 1, inplace = True)

In [24]:
# save out as geoparquet 
shared_utils.utils.geoparquet_gcs_export(bridges_intersect_dissolve, GCS_FILE_PATH, "bridgeareas_clean")